In [1]:
import itertools
import string
from collections import defaultdict
from pathlib import Path

import numpy as np
import pandas as pd

In [2]:
RAW_FILE   = "originals/Raw international financial data_FY22-25.xlsx"
RAW_SHEET  = "FY25 Mar Est - Europe (2)"

REF_FILE   = "originals/CLN_FY25_International - products ONLY.xlsx"
REF_SHEET  = "CLN - EU - Products ONLY"

OUTPUT_DIR   = "outputs"
OUTPUT_CSV   = f"{OUTPUT_DIR}/cleaned_products_eu.csv"
OUTPUT_XLSX  = f"{OUTPUT_DIR}/cleaned_products_eu.xlsx"
OUTPUT_SHEET = "AUTO_CLEAN"

RENAME = {
    "Licensee": "Partner",
    "Product Category": "Category",
    "Primary Territory": "Primary territory",
    "SW #": "Combined SW #",
    "Net Revenue": "Net revenue",
    "Start Date": "Start date",
    "End Date": "End date",
}

BRACKETS = [
    (25, "0-25K"),
    (49, "26-49K"),
    (100, "50-100K"),
    (249, "101-249K"),
    (499, "250-499K"),
    (float("inf"), "500+"),
]
BINS, LABELS = zip(*BRACKETS, strict=False)

FINAL_COLS = [
    "Region",
    "LOB",
    "Combined SW #",
    "Partner",
    "Category",
    "Primary territory",
    "Start date",
    "End date",
    "Bracket",
    "Lifecycle",
    "High / Med/Low touch",
    "Details",
    "Net revenue",
    *LABELS,
]

In [3]:
def detect_header(sheet: str, marker: str = "Sub Dept", n_check: int = 40) -> int:
    """Return row index of the real header (first col contains *marker*)."""
    sample = pd.read_excel(RAW_FILE, sheet, header=None, nrows=n_check)
    row = sample.index[sample.iloc[:, 0].astype(str).str.contains(marker, na=False)]
    if row.empty:
        raise ValueError("Could not find header row")
    return int(row[0])


def load_raw() -> pd.DataFrame:
    hdr = detect_header(RAW_SHEET)
    df  = pd.read_excel(RAW_FILE, RAW_SHEET, header=hdr)

    # trim whitespace
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

    # *** NEW: keep rows that have a real Product Category
    df = df[df["Product Category"].notna() & (df["Product Category"].str.len() > 0)]

    # keep product lines only (exclude totals)
    mask = (
        df["Sub Dept"].str.contains("Product", case=False, na=False)
        & ~df["Sub Dept"].str.contains("Total", case=False, na=False)
    )
    return df.loc[mask].copy()

# def aggregate_rows(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Collapse exact duplicates *including territory* to avoid
#     double-counting while keeping the one-territory-per-row rule.
#     """
#     key = [
#         "Partner",
#         "Combined SW #",
#         "Category",
#         "Primary territory",
#         "Start date",
#         "End date",
#     ]
#     return (
#         df.groupby(key, dropna=False, as_index=False).agg(
#             {"Net revenue": "sum"}
#         )
#     )

def aggregate_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Roll-up exact duplicates, **except** leave each “New Business” /
    placeholder deal as its own commercial line (only collapse if the
    FINANCE sheet literally duplicated the same territory row).
    """
    base_key = ["Partner", "Combined SW #", "Category",
                "Start date", "End date"]

    nb_mask  = df["Partner"].str.lower() == "new business"
    regular  = df.loc[~nb_mask]
    newbiz   = df.loc[ nb_mask]

    # ―― normal roll-up (territories concatenated) ―――――――――――――――――――
    regular_out = (
        regular.groupby(base_key, dropna=False, as_index=False)
               .agg({"Net revenue": "sum",
                     "Primary territory":
                         lambda s: ", ".join(sorted(set(s.dropna())))})
    )

    # ―― “New Business”: keep one row per territory ―――――――――――――――――――
    nb_key = base_key + ["Primary territory"]
    newbiz_out = (
        newbiz.groupby(nb_key, dropna=False, as_index=False)
               .agg({"Net revenue": "sum"})
    )

    return pd.concat([regular_out, newbiz_out], ignore_index=True)

def _split_territories(df: pd.DataFrame) -> pd.DataFrame:
    """Each territory gets its own row, revenue is divided equally.
       The original aggregated row is DROPPED to avoid double-counting."""
    rows = []
    for _, r in df.iterrows():
        terr_raw = str(r["Primary territory"])
        parts = [t.strip() for t in terr_raw.split(",") if t.strip()]
        if len(parts) <= 1:
            rows.append(r)
            continue

        # divide revenue equally (keeps grand-total unchanged)
        share = r["Net revenue"] / len(parts) if pd.notnull(r["Net revenue"]) else np.nan
        for terr in parts:
            nr          = r.copy()
            nr["Primary territory"] = terr
            nr["Net revenue"]       = share
            rows.append(nr)

    return pd.DataFrame(rows)

def rollup_fx(df: pd.DataFrame) -> pd.DataFrame:
    """Combine all 'FX Gain/Loss' lines into a single one."""
    mask = df["Partner"].str.contains("FX Gain", case=False, na=False)
    if mask.sum() <= 1:
        return df

    total = df.loc[mask, "Net revenue"].sum()
    # use the *first* FX row as template
    row = df.loc[mask].iloc[0].copy()
    row["Net revenue"] = total

    keep = df.loc[~mask]
    return pd.concat([keep, row.to_frame().T], ignore_index=True)


def assign_ids(df: pd.DataFrame) -> pd.DataFrame:
    id_counter = defaultdict(int)
    blank_it   = itertools.count(1)
    tbd_it     = itertools.count(1)
    new_ids    = []

    for base_raw, partner in zip(df['_base'], df['Partner'], strict=False):
        base = str(base_raw).strip().upper()

        # -------- placeholder logic ------------------------------------
        if base in {'', 'NAN', 'NONE', 'TB', 'TBD'}:
            if str(partner).strip().lower() == 'new business':
                new_ids.append(f'Blank {next(blank_it)}')
            elif str(partner).lower().startswith('fx gain'):
                new_ids.append('FX Gain(Loss)')
            else:
                new_ids.append(f'TBD{next(tbd_it)}')
            continue

        # real SW number → apply A/B/C suffix
        seq = id_counter[base]
        suffix = "" if seq == 0 else string.ascii_uppercase[seq - 1]
        new_ids.append(f"{base}{suffix}")
        id_counter[base] += 1

    df["Combined SW #"] = new_ids
    return df

def add_brackets(df: pd.DataFrame) -> pd.DataFrame:
    df["Bracket"] = pd.cut(df["Net revenue"], bins=[-np.inf, *BINS], labels=LABELS)
    for lab in LABELS:
        df[lab] = (df["Bracket"] == lab).astype(int)
    return df

In [4]:
raw = load_raw().rename(columns=RENAME)

# ensure date dtype
for c in ("Start date", "End date"):
    raw[c] = pd.to_datetime(raw[c], errors="coerce")

# group duplicates (territory‑aware)
df = aggregate_rows(raw)
df = _split_territories(df)

# static columns
df["Region"] = "EUROPE"
df["LOB"] = "Products"

# prepare for ID assignment
df["_base"] = (
    df["Combined SW #"]
    .fillna("")
    .astype(str)
    .str.replace(r"[A-Z]$", "", regex=True)
)
df = df.sort_values(["_base", "Partner", "Primary territory"])

df = assign_ids(df)

# empty columns that will be filled manually later
for col in ("Lifecycle", "High / Med/Low touch", "Details"):
    df[col] = ""

df = rollup_fx(df)
df = add_brackets(df)

out = df[FINAL_COLS]

# -------------------------- write files
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
out.to_csv(OUTPUT_CSV, index=False)
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as xls:
    out.to_excel(xls, sheet_name=OUTPUT_SHEET, index=False)

print("Saved ➜", OUTPUT_CSV, "and", OUTPUT_XLSX)


Saved ➜ outputs/cleaned_products_eu.csv and outputs/cleaned_products_eu.xlsx


/var/folders/_y/42hkq40978gd7mrq8g5yrw1m0000gn/T/ipykernel_84113/3863647478.py:110: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([keep, row.to_frame().T], ignore_index=True)


In [5]:
if Path(REF_FILE).exists():
    ref = pd.read_excel(REF_FILE, REF_SHEET, header=2)
    ref = ref.loc[:, ~ref.columns.str.startswith("Unnamed")]

    print("\n— sanity —")
    print("rows   ref / ours:", ref.shape[0], "/", out.shape[0])
    print(
        "total €k       :",
        round(ref["Net revenue"].sum() / 1_000, 1),
        "/",
        round(out["Net revenue"].sum() / 1_000, 1),
    )
    print("dup IDs        :", out["Combined SW #"].duplicated().sum())


— sanity —
rows   ref / ours: 126 / 163
total €k       : 1.6 / 1.6
dup IDs        : 0


/Users/idinu/Documents/Work/Sesame/deliverables/jess-lob/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
